Once you installed the package, run the command mlflow --version and check the output.



In [1]:
import mlflow

print(mlflow.__version__)

OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k
OpenBLAS WARNING - could not determine the L2 cache size on this system, assuming 256k


2.12.2


In [2]:
import os

[i for i in os.listdir('/srv/data/trips') if ('parquet' in i and '2023' in i)]

['green_tripdata_2023-01.parquet',
 'yellow_tripdata_2023-01.parquet',
 'yellow_tripdata_2023-02.parquet',
 'green_tripdata_2023-02.parquet',
 'green_tripdata_2023-03.parquet']

Change dir

```shell
cd src/jupyter_notebooks
```

Script

```shell
python preprocess_data.py --raw_data_path /srv/data/trips --dest_path ./output
```

In [3]:
print('Num files in dir: %d' % len(os.listdir('output')))

os.listdir('output')

Num files in dir: 4


['train.pkl', 'val.pkl', 'dv.pkl', 'test.pkl']

In [5]:
import os
import pickle
import mlflow

from mlflow.entities import ViewType
from mlflow.tracking import MlflowClient
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

HPO_EXPERIMENT_NAME = "random-forest-hyperopt"
EXPERIMENT_NAME = "random-forest-best-models"
RF_PARAMS = ['max_depth', 'n_estimators', 'min_samples_split', 'min_samples_leaf', 'random_state']

mlflow.set_tracking_uri("http://mlflow_container_ui:8000")

client = MlflowClient()

experiment = client.get_experiment_by_name(HPO_EXPERIMENT_NAME)
runs = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.rmse ASC"]
)
for run in runs:
    print(run.data.params)

{'max_depth': '19', 'min_samples_leaf': '2', 'min_samples_split': '2', 'n_estimators': '11', 'random_state': '42'}
{'max_depth': '15', 'min_samples_leaf': '2', 'min_samples_split': '3', 'n_estimators': '40', 'random_state': '42'}
{'max_depth': '20', 'min_samples_leaf': '1', 'min_samples_split': '9', 'n_estimators': '19', 'random_state': '42'}
{'max_depth': '14', 'min_samples_leaf': '3', 'min_samples_split': '4', 'n_estimators': '26', 'random_state': '42'}
{'max_depth': '14', 'min_samples_leaf': '2', 'min_samples_split': '6', 'n_estimators': '23', 'random_state': '42'}


In [13]:
EXPERIMENT_NAME = "random-forest-best-models"
experiment = client.get_experiment_by_name(EXPERIMENT_NAME)
run = client.search_runs(
    experiment_ids=experiment.experiment_id,
    run_view_type=ViewType.ACTIVE_ONLY,
    max_results=5,
    order_by=["metrics.test_rmse ASC"]
)[0]

f'runs:/{run.info.run_id}/model'

'runs:/3bcd2305a0794b8c9870e116edf6616d/model'